In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
features = pd.read_csv("data/features.csv", delimiter="|")
historic_sales = pd.read_csv("data/historic_sales.csv", delimiter="|")
store_info = pd.read_csv("data/store_info.csv", delimiter=";")

In [ ]:
features = features.drop("Unnamed: 0", axis=1)
historic_sales = historic_sales.drop("Unnamed: 0", axis=1)
store_info = store_info.drop("Unnamed: 0", axis=1)

In [ ]:
features["Date"] = pd.to_datetime(features["Date"])

In [ ]:
historic_sales = historic_sales.sort_values(by=['Store', 'Dept', 'Date'])

In [ ]:
idx = (historic_sales["Dept"] == '2011-04-15')

In [ ]:
idx = (historic_sales["Dept"] == '2011-04-15')
historic_sales.loc[idx,['Dept','Date', 'IsHoliday', 'Weekly_Sales']] = historic_sales.loc[idx,['Date','Dept', 'Weekly_Sales', 'IsHoliday']].values

In [ ]:
historic_sales["Dept"] = historic_sales["Dept"].apply(lambda x: int(float(x.replace(",","."))) if type(x) is str else x).astype('category')

In [ ]:
historic_sales["Weekly_Sales"] = pd.to_numeric(historic_sales["Weekly_Sales"].apply(lambda x: x.replace("nan","").replace(",",".") if type(x) is str else x ))

In [ ]:
historic_sales.loc[(historic_sales["Weekly_Sales"]<0), "Weekly_Sales"] = np.nan

In [ ]:
def fillFirstLast(df):
    for s in df["Store"].unique():
        for d in df[df["Store"]==s]["Dept"].unique():

            toFill = df[(df["Store"]==s) & (df["Dept"]==d)]["Weekly_Sales"].isnull()

            i = 0
            bfiller = None
            index = None
            while i < len(toFill)-1 and toFill.iloc[i]:
                i += 1
                bfiller = df[(df["Store"]==s) & (df["Dept"]==d)]["Weekly_Sales"].iloc[i]
                index = df[(df["Store"]==s) & (df["Dept"]==d)].index[i]


            if bfiller != None: 
                for j in range(index-i, index):
                    df.loc[j, "Weekly_Sales"] = bfiller

            i = len(toFill)-1
            ffiller = None
            index = None
            while i > 0 and toFill.iloc[i]:
                i -= 1
                ffiller = df[(df["Store"]==s) & (df["Dept"]==d)]["Weekly_Sales"].iloc[i]
                index = df[(df["Store"]==s) & (df["Dept"]==d)].index[i]

            diff = len(toFill)-1-i
            if ffiller != None: 
                for j in range(index+1, index+diff+1):
                    df.loc[j, "Weekly_Sales"] = ffiller
    return df

historic_sales = fillFirstLast(historic_sales)
            

In [ ]:
historic_sales["Weekly_Sales"] = (historic_sales["Weekly_Sales"].fillna(method='ffill') + historic_sales["Weekly_Sales"].fillna(method='bfill'))/2

In [ ]:
historic_sales["IsHoliday"] = historic_sales["IsHoliday"].astype('bool')

In [ ]:
historic_sales["Date"] = pd.to_datetime(historic_sales["Date"])

In [ ]:
historic_sales = historic_sales.drop(193967)

In [ ]:
from datetime import timedelta

In [ ]:
hs_ewma_ctr = historic_sales.groupby(["Store", "Dept"]).agg({'Date': ['min','max']}).reset_index()

In [ ]:
hs_ewma =  historic_sales.loc[:, ('Store', 'Dept', 'Date', 'Weekly_Sales')]

In [ ]:
missings = {'Store' : [], 'Dept' : [], 'Date' : [], 'Weekly_Sales' : []}
for s in hs_ewma_ctr["Store"].unique():
    for d in hs_ewma_ctr[hs_ewma_ctr["Store"]==s]["Dept"].unique():
        idx = (hs_ewma_ctr["Store"]==s)  & (hs_ewma_ctr["Dept"]==d)
        current = hs_ewma_ctr[idx]['Date']['min'].iloc[0]
        last = hs_ewma_ctr[idx]['Date']['max'].iloc[0]
        existing = historic_sales.loc[(historic_sales["Store"]==s)  & (historic_sales["Dept"]==d), "Date"]
        #print(existing)
        while current <= last:
            if (existing == current).sum() == 0:
                missings['Store'].append(s)
                missings['Dept'].append(d)
                missings['Date'].append(current)
                missings['Weekly_Sales'].append(None)
                #print(s, d, current)
            current += timedelta(days=7)


In [ ]:
missings = pd.DataFrame(missings)

In [ ]:
missings["Weekly_Sales"] = pd.to_numeric(missings["Weekly_Sales"])
missings["Dept"] = missings["Dept"].astype('category')
missings["Store"] = missings["Store"].astype('category')

In [ ]:
hs_ewma = hs_ewma.append(missings)

In [ ]:
hs_ewma = hs_ewma.sort_values(by=['Store', 'Dept', 'Date'])

In [ ]:
hs_ewma.reset_index(drop=True, inplace=True)

In [ ]:
hs_ewma["Weekly_Sales"] = (hs_ewma["Weekly_Sales"].fillna(method='ffill') + hs_ewma["Weekly_Sales"].fillna(method='bfill'))/2

In [ ]:
grouped = hs_ewma.groupby(["Store", "Dept"])

In [ ]:
hs_ewma["hist_1y"] = grouped["Weekly_Sales"].shift(52)
hs_ewma["hist_1w"] = grouped["Weekly_Sales"].shift()

In [ ]:
grouped = hs_ewma.groupby(["Store", "Dept"])

In [ ]:
hs_ewma["hist_2w"] = grouped["hist_1w"].shift()

In [ ]:
hs_ewma

In [ ]:
grouped["hist_1w"].ewm(span=4).mean().index.get_level_values(2).duplicated().any()

In [ ]:
spans = [4, 13, 52]
cols = ['ewma_1m', 'ewma_3m', 'ewma_1y']

for span, col in zip(spans, cols):

    data = grouped["hist_1w"].ewm(span=span).mean()
    data.index = data.index.get_level_values(2)
    hs_ewma[col] = data

In [ ]:
hs_ewma["diff_1w"] = grouped["hist_1w"].diff()
hs_ewma["diff_2w"] = grouped["hist_2w"].diff()
hs_ewma["diff_1y"] = grouped["hist_1w"].diff(52)

In [ ]:
spans = [4, 13, 52]
cols = ['std_1m', 'std_3m', 'std_1y']

for span, col in zip(spans, cols):

    data = grouped["hist_1w"].rolling(span).std()
    data.index = data.index.get_level_values(2)
    hs_ewma[col] = data

In [ ]:
grouped = hs_ewma.groupby(["Store", "Dept"])

In [ ]:
store_info["Type"] = store_info["Type"].astype("category")

In [ ]:
historic_sales_store_info = historic_sales.merge(store_info, how="left", on="Store")

In [ ]:
df = historic_sales_store_info.merge(features, how="left", on=("Store", "Date"))

In [ ]:
df = df.merge(hs_ewma, how="left", on=("Store", "Dept", "Date"))

In [ ]:
df["Weekly_Sales"] = df["Weekly_Sales_y"]
df["IsHoliday"] = df["IsHoliday_x"]
df = df.drop(labels=['IsHoliday_x', 'IsHoliday_y', 'Weekly_Sales_x', 'Weekly_Sales_y'], axis=1)

In [ ]:
df["hasMD1"] = df["MarkDown1"].isnull().apply(lambda x : not(x))
df["hasMD2"] = df["MarkDown2"].isnull().apply(lambda x : not(x))
df["hasMD3"] = df["MarkDown3"].isnull().apply(lambda x : not(x))
df["hasMD4"] = df["MarkDown4"].isnull().apply(lambda x : not(x))
df["hasMD5"] = df["MarkDown5"].isnull().apply(lambda x : not(x))

In [ ]:
df["MarkDown1"] = df["MarkDown1"].fillna(0)
df["MarkDown2"] = df["MarkDown2"].fillna(0)
df["MarkDown3"] = df["MarkDown3"].fillna(0)
df["MarkDown4"] = df["MarkDown4"].fillna(0)
df["MarkDown5"] = df["MarkDown5"].fillna(0)

In [ ]:
df = pd.concat([df, pd.get_dummies(df['Dept'], prefix="D")], axis=1)

In [ ]:
df = pd.concat([df, pd.get_dummies(df['Type'])], axis=1)

In [ ]:
df = pd.concat([df, pd.get_dummies(df['Store'], prefix="S")], axis=1)

In [ ]:
df["Month"] = df["Date"].apply(lambda x : x.month)

In [ ]:
df["Month"] = df["Month"].astype('category')

In [ ]:
df = pd.concat([df, pd.get_dummies(df['Month'], prefix="M")], axis=1)

In [ ]:
df = df.dropna()

In [ ]:
def trainTest(df, n):
    grouped = df.groupby(["Store", "Dept"])
    test = grouped.tail(n)
    test.loc[:, "Weekly_Sales_True"] = test["Weekly_Sales"]
    test.loc[:, "Weekly_Sales"] = np.nan
    
    grouped = df.groupby(["Store", "Dept"], as_index=False)
    train = grouped.apply(lambda x: x.iloc[:-n])
    train.index = train.index.droplevel()
    
    return train, test

In [ ]:
train, test = trainTest(df, 1)

In [ ]:
model = GradientBoostingRegressor(random_state=0, n_estimators=100, max_depth = 3, verbose = 1)

In [ ]:
outcome_var = 'Weekly_Sales'

In [ ]:
predictor_var = ['Size', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'ewma_1m', 'ewma_3m', 'ewma_1y', 'hist_1y', 'hist_1w', 'diff_1w', 'diff_1y', 'std_1m', 'std_3m', 'std_1y', 'IsHoliday', 'hasMD1', 'hasMD2', 'hasMD3', 'hasMD4', 'hasMD5', 'D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10', 'D_11', 'D_12', 'D_13', 'D_14', 'D_16', 'D_17', 'D_18', 'D_19', 'D_20', 'D_21', 'D_22', 'D_23', 'D_24', 'D_25', 'D_26', 'D_27', 'D_28', 'D_29', 'D_30', 'D_31', 'D_32', 'D_33', 'D_34', 'D_35', 'D_36', 'D_37', 'D_38', 'D_40', 'D_41', 'D_42', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_65', 'D_67', 'D_71', 'D_72', 'D_74', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83', 'D_85', 'D_87', 'D_90', 'D_91', 'D_92', 'D_93', 'D_94', 'D_95', 'D_96', 'D_97', 'D_98', 'D_99', 'A', 'B', 'C', 'S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 'S_10', 'S_11', 'S_12', 'S_13', 'S_14', 'S_15', 'S_16', 'S_17', 'S_18', 'S_19', 'S_20', 'S_21', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27', 'S_28', 'S_29', 'S_30', 'S_31', 'S_32', 'S_33', 'S_34', 'S_35', 'S_36', 'S_37', 'S_38', 'S_39', 'S_40', 'S_41', 'S_42', 'S_43', 'S_44', 'S_45', 'M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6', 'M_7', 'M_8', 'M_9', 'M_10', 'M_11', 'M_12']

In [ ]:
model.fit(df[predictor_var],df[outcome_var])

In [ ]:
df["Weekly_Sales_Pred"] = model.predict(df[predictor_var])
print(r2_score(df["Weekly_Sales"], df["Weekly_Sales_Pred"]))
print(mean_squared_error(df["Weekly_Sales"], df["Weekly_Sales_Pred"])**0.5)
print(mean_absolute_error(df["Weekly_Sales"], df["Weekly_Sales_Pred"]))

In [ ]:
model.fit(train[predictor_var],train[outcome_var])

In [ ]:
test["Weekly_Sales"] = model.predict(test[predictor_var])
print(r2_score(test["Weekly_Sales_True"], test["Weekly_Sales"]))
print(mean_squared_error(test["Weekly_Sales_True"], test["Weekly_Sales"])**0.5)
print(mean_absolute_error(test["Weekly_Sales_True"], test["Weekly_Sales"])**0.5)